# Index Return data

- using yahoo finance for historical stock return data 
- going to need the following
    - Dates of the announcement
    - Index returns 10 days before the announcement
    - Index returns the day of the announcement
    - Index returns 10 days after the announcement 

In [20]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta


In [22]:
fomc_statements = pd.read_csv('dates/fomc_statements.csv')
fomc_statements['Date'] = pd.to_datetime(fomc_statements['Date'])

tickers = ['^IXIC', 'XLRE', '^GSPC', '^RUT', '^W5000', '^DJI', 'XLU', 'XLF', 'XLY']
start_date = fomc_statements['Date'].min() - pd.Timedelta(days=15)
end_date = fomc_statements['Date'].max() + pd.Timedelta(days=15)

all_indices_data = {}
for ticker in tickers:
    print(f"Downloading data for {ticker}...")
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = data.columns.get_level_values(0)
    data['return'] = data['Close'].pct_change()  
    all_indices_data[ticker] = data[['return']].dropna()
    print(f"Data for {ticker} downloaded.")

rows = []

for date in fomc_statements['Date']:
    for ticker in tickers:
        row = {'announcement_date': date, 'ticker': ticker}
        for t in range(-10, 11):
            target_date = date + pd.Timedelta(days=t)
            if target_date in all_indices_data[ticker].index:
                row[f'T{t:+}'] = all_indices_data[ticker].loc[target_date, 'return']
            else:
                row[f'T{t:+}'] = pd.NA
        rows.append(row)

final_df = pd.DataFrame(rows)

column_order = ['announcement_date', 'ticker'] + [f'T{t:+}' for t in range(-10, 11)]
final_df = final_df[column_order]

final_df

# final_df.to_csv('long_format_output.csv', index=False)

/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_90902/925711554.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fomc_statements['Date'] = pd.to_datetime(fomc_statements['Date'])
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Data for ^IXIC downloaded.
Data for XLRE downloaded.
Data for ^GSPC downloaded.
Data for ^RUT downloaded.


[*********************100%***********************]  1 of 1 completed


Data for ^W5000 downloaded.


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


Data for XLF downloaded.


[*********************100%***********************]  1 of 1 completed


Data for XLY downloaded.


,announcement_date,ticker,T-10,T-9,T-8,T-7,T-6,T-5,T-4,T-3,...,T+1,T+2,T+3,T+4,T+5,T+6,T+7,T+8,T+9,T+10
0,2000-02-02,^IXIC,<NA>,-0.032894,0.017414,-0.023396,-0.007457,-0.037749,<NA>,<NA>,...,0.033633,0.007875,<NA>,<NA>,0.018291,0.024465,-0.014514,0.02805,-0.020104,<NA>
1,2000-02-02,XLRE,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2000-02-02,^GSPC,<NA>,-0.027634,0.006065,-0.004213,-0.003938,-0.027457,<NA>,<NA>,...,0.011248,-0.000421,<NA>,<NA>,-0.000091,0.012273,-0.020815,0.003627,-0.020969,<NA>
3,2000-02-02,^RUT,<NA>,-0.020583,-0.002601,-0.001055,-0.007715,-0.023984,<NA>,<NA>,...,0.023025,0.007457,<NA>,<NA>,0.013073,0.009579,-0.002772,0.011586,-0.009424,<NA>
4,2000-02-02,^W5000,<NA>,-0.026098,0.005497,-0.004102,-0.004174,-0.028225,<NA>,<NA>,...,0.014137,-0.000158,<NA>,<NA>,0.002874,0.012514,-0.018755,0.005698,-0.018061,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1849,2025-03-19,^W5000,<NA>,-0.027938,-0.006469,0.004868,-0.01474,0.021989,<NA>,<NA>,...,-0.002582,0.000463,<NA>,<NA>,0.019161,0.000932,-0.011556,-0.004142,-0.020066,<NA>
1850,2025-03-19,^DJI,<NA>,-0.020794,-0.01141,-0.001992,-0.012995,0.016529,<NA>,<NA>,...,-0.000269,0.000763,<NA>,<NA>,0.014242,0.000098,-0.003116,-0.003653,-0.016922,<NA>
1851,2025-03-19,XLU,<NA>,0.010998,-0.009342,-0.0031,0.002462,0.019519,<NA>,<NA>,...,0.004054,-0.006435,<NA>,<NA>,-0.00032,-0.015991,0.00663,-0.000258,0.007363,<NA>
1852,2025-03-19,XLF,<NA>,-0.022394,-0.008538,0.00147,-0.005453,0.023197,<NA>,<NA>,...,0.001412,-0.003626,<NA>,<NA>,0.017634,0.005583,-0.002974,-0.001989,-0.01933,<NA>
